In [ ]:
# Modify environment variables accordingly
%set_env ENV_ARCHIVE_RELPATH=MyDrive/Motorage/AnnotatedLicensePlates.zip
%set_env ENV_BASE_DRIVE_ABSPATH=/content/drive

%set_env ENV_DATASET_ABSPATH=/content/datasets/license_plate
%set_env ENV_ANNOTATIONS_DIRECTORY_NAME=annotations
%set_env ENV_LABELS_DIRECTORY_NAME=labels
%set_env ENV_IMAGES_DIRECTORY_NAME=images

%set_env ENV_CLASS_MAPPING_FILE_NAME=class_mapping.csv 
%set_env ENV_DATA_YAML_FILE_NAME=data.yaml

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!mkdir -p $ENV_DATASET_ABSPATH
!unzip -q $ENV_BASE_DRIVE_ABSPATH/$ENV_ARCHIVE_RELPATH -d $ENV_DATASET_ABSPATH
# Rename annotations directory into labels directory (standard name for YOLOv5)
!mv $ENV_DATASET_ABSPATH/$ENV_ANNOTATIONS_DIRECTORY_NAME $ENV_DATASET_ABSPATH/$ENV_LABELS_DIRECTORY_NAME

In [ ]:
# import torch
# torch.hub.download_url_to_file('<url>', '<archive_name>')

In [3]:
import os

DATASET_DIR_PATH = os.environ['ENV_DATASET_ABSPATH']
LABELS_DIR_PATH = os.path.join(DATASET_DIR_PATH, os.environ['ENV_LABELS_DIRECTORY_NAME'])
IMAGES_DIR_PATH = os.path.join(DATASET_DIR_PATH, os.environ['ENV_IMAGES_DIRECTORY_NAME'])

images = sorted([os.path.join(IMAGES_DIR_PATH, file_name) for file_name in os.listdir(IMAGES_DIR_PATH)])
labels = sorted([os.path.join(LABELS_DIR_PATH, file_name) for file_name in os.listdir(LABELS_DIR_PATH)])

# Verify dataset naming validity
assert len(images) == len(labels), f"File lengths do not match: {len(images)} and {len(labels)}"
for idx in range(len(images)):
  image_name = os.path.splitext(os.path.basename(images[idx]))[0]
  label_name = os.path.splitext(os.path.basename(labels[idx]))[0]
  assert image_name == label_name , f"Names do not match: {image_name} and {label_name}"

In [4]:
from sklearn.model_selection import train_test_split

# Split dataset into <train, valid, test> splits 
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size = 0.2, random_state = 1)
val_images, test_images, val_labels, test_labels = train_test_split(val_images, val_labels, test_size = 0.5, random_state = 1)

In [5]:
import shutil, os

def relocate_files(target_dir_path: str, files: list):
  if not os.path.exists(target_dir_path):
    os.makedirs(target_dir_path)
  for file in files: 
    shutil.move(file, target_dir_path)

TRAIN_DIR_NAME = "train"
VAL_DIR_NAME = "val"
TEST_DIR_NAME = "test"

# Relocate splits to standard YOLOv5 directories
relocation_mapping = {
    os.path.join(IMAGES_DIR_PATH, TRAIN_DIR_NAME): train_images,
    os.path.join(IMAGES_DIR_PATH, VAL_DIR_NAME): val_images,
    os.path.join(IMAGES_DIR_PATH, TEST_DIR_NAME): test_images,
    os.path.join(LABELS_DIR_PATH, TRAIN_DIR_NAME): train_labels,
    os.path.join(LABELS_DIR_PATH, VAL_DIR_NAME): val_labels,
    os.path.join(LABELS_DIR_PATH, TEST_DIR_NAME): test_labels,
}

for target_dir, files in relocation_mapping.items():
  relocate_files(target_dir, files)

In [6]:
CLASS_MAPPING_FILE_NAME = os.environ.get('ENV_CLASS_MAPPING_FILE_NAME')
DATA_YAML_FILE_NAME = os.environ.get('ENV_DATA_YAML_FILE_NAME')

yaml_mapping = {
    TRAIN_DIR_NAME: os.path.join(IMAGES_DIR_PATH, TRAIN_DIR_NAME),
    VAL_DIR_NAME: os.path.join(IMAGES_DIR_PATH, VAL_DIR_NAME),
    TEST_DIR_NAME: os.path.join(IMAGES_DIR_PATH, TEST_DIR_NAME)
}

with open(os.path.join(DATASET_DIR_PATH, CLASS_MAPPING_FILE_NAME)) as mapping_file:
  nested_label_names = [line.strip().split(",") for line in mapping_file.readlines()]
  annotation_labels = {key: value for key, value in nested_label_names}
  output_buffer = [f'{key}: {value}' for key, value in yaml_mapping.items()]
  output_buffer += ["nc: " + str(len(annotation_labels)), "names: " + str(list(annotation_labels.values()))]
  print("\n".join(output_buffer), file=open(os.path.join(DATASET_DIR_PATH, DATA_YAML_FILE_NAME), 'w'))

!cat $ENV_DATASET_ABSPATH/$ENV_DATA_YAML_FILE_NAME

train: /content/datasets/license_plate/images/train
val: /content/datasets/license_plate/images/val
test: /content/datasets/license_plate/images/test
nc: 1
names: ['licence']


In [ ]:
%cd yolov5
!pip install -r --quiet requirements.txt

In [ ]:
!python train.py --img 640 --batch 32 --epochs 100 --data $ENV_DATASET_ABSPATH/$ENV_DATA_YAML_FILE_NAME --weights yolov5s.pt --workers 24

In [ ]:
!python detect.py --source $ENV_DATASET_ABSPATH/$ENV_IMAGES_DIRECTORY_NAME/test --weights runs/train/exp/weights/best.pt --conf 0.25

In [ ]:
from PIL import Image

DETECTIONS_DIR = input("Provide path to local (Google Colab) detection directory\n")
assert os.path.exists(DETECTIONS_DIR), "Path provided does not exist"
detection_images = [os.path.join(DETECTIONS_DIR, file_name) for file_name in os.listdir(DETECTIONS_DIR)]

for image in detection_images:
  display(Image.open(image))

In [ ]:
import os, shutil

# Save weights to drive for further use

DRIVE_ABS_PATH = os.environ.get('ENV_BASE_DRIVE_ABSPATH')
ARCHIVE_REL_PATH = os.environ.get('ENV_ARCHIVE_RELPATH')
OUTPUT_DIRECTORY_PATH = os.path.join(DRIVE_ABS_PATH, os.path.splitext(ARCHIVE_REL_PATH)[0])
LOCAL_WEIGHTS_FILE_PATH = input("Provide path to local (Google Colab) weights\n")

assert os.path.exists(LOCAL_WEIGHTS_FILE_PATH), "Path provided does not exist"

if not os.path.exists(OUTPUT_DIRECTORY_PATH):
  os.makedirs(OUTPUT_DIRECTORY_PATH)
shutil.copy(LOCAL_WEIGHTS_FILE_PATH, OUTPUT_DIRECTORY_PATH)

In [ ]:
%%writefile yolov5cfg.yaml

# Optionally, tune hyperparameters and retrain

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)